### Name:
> Castillo, Marvien Angel C. <br>
> Herrera, Mikhaela Gabrielle B. <br>
> Regindin, Sean Adrien I. <br>

# Setup Environment

In [1]:
import os

# Add the directory containing the executable to the PATH
os.environ["PATH"] += os.pathsep + "/usr/local/cuda/bin"

# Check if the directory is added to the PATH
print(os.environ["PATH"])

/opt/tljh/user/bin:/bin:/usr/bin:/usr/local/cuda/bin


# Check if CUDA is present

In [2]:
%%bash
nvcc --version
nvprof --version
nsys --version
ncu --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2025 NVIDIA Corporation
Built on Wed_Apr__9_19:24:57_PDT_2025
Cuda compilation tools, release 12.9, V12.9.41
Build cuda_12.9.r12.9/compiler.35813241_0
nvprof: NVIDIA (R) Cuda command line profiler
Copyright (c) 2012 - 2025 NVIDIA Corporation
Release version 12.9.19 (21)
NVIDIA Nsight Systems version 2025.1.3.140-251335620677v0
NVIDIA (R) Nsight Compute Command Line Profiler
Copyright (c) 2018-2025 NVIDIA Corporation
Version 2025.2.0.0 (build 35613519) (public-release)


In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Nov  6 08:25:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 575.51.03              Driver Version: 575.51.03      CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-PCIE-32GB           Off |   00000000:00:10.0 Off |                    0 |
| N/A   26C    P0             22W /  250W |       0MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Variant 1 - C Program

In [4]:
%%writefile C_var1.c

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>
// ***C function version
void kernel_C(float A[], float B[], float C[], size_t n, int idx[]) {
	for (int i = 0; i < n; i++) {
		if (A[i] >= B[i]) {
			C[i] = A[i];
			idx[i] = 0;
		}
		else {
			C[i] = B[i];
			idx[i] = 1;
		}
	}
}

int main(int argc, char** argv)
{
   const size_t ARRAY_SIZE = 1<<24;
   const size_t INT_ARRAY_BYTES = ARRAY_SIZE * sizeof(int);
   const size_t FLOAT_ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
//number of times the program is to be executed
   const size_t loope = 10;
//declare array
   float *C,*A,*B;
   int *idx,a;
   A = (float*)malloc(FLOAT_ARRAY_BYTES);
   B = (float*)malloc(FLOAT_ARRAY_BYTES);
   C = (float*)malloc(FLOAT_ARRAY_BYTES);
   idx = (int*)malloc(INT_ARRAY_BYTES);
   a=2;
//timer variables
  clock_t start, end;
// ***--- initialize your array here ---------
   int i;
	for (i = 0; i < ARRAY_SIZE; i++) {
		A[i] = sin(i * 0.0005) * 100.0 + 50.0;
		B[i] = cos(i * 0.0003) * 100.0 + 50.0;
	}
// fill-in cache
    kernel_C(A,B,C,ARRAY_SIZE,idx);
//time here
  double elapse, time_taken;
  elapse = 0.0f;
  for (int i=0; i<loope; i++){
    start = clock();
      kernel_C(A,B,C,ARRAY_SIZE,idx );
    end = clock();
    time_taken = ((double)(end-start))*1E3/CLOCKS_PER_SEC;
    elapse = elapse + time_taken;
  }
  printf("Function (in C) average time for %lu loops is %f milliseconds to execute an array size %lu \n", loope, elapse/loope, ARRAY_SIZE);

// error checking routine here --
   size_t err_count = 0;
   int sanity_checker = 0;
   for (int i = 0; i < ARRAY_SIZE; i++) { 
        float expected_C = (A[i] >= B[i]) ? A[i] : B[i];
        int expected_idx = (A[i] >= B[i]) ? 0 : 1;
    
        if (fabs(C[i] - expected_C) > 1e-5 || idx[i] != expected_idx) {
            sanity_checker++;
        }
    }
  printf("Sanity Checker = %d",sanity_checker);
  
  // Free memory
  free(A);
  free(B);
  free(C);
  free(idx);
  return 0;
}

Overwriting C_var1.c


In [5]:
%%bash
gcc C_var1.c -o C_var1 -lm

In [6]:
%%bash
./C_var1

Function (in C) average time for 10 loops is 287.938900 milliseconds to execute an array size 16777216 
Sanity Checker = 0

# Variant 2 - Grid Stride Loop

In [7]:
%%writefile CUDA_var2.cu

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
//Grid stride loop

//*** CUDA kernel
__global__
void kernel_C(size_t n, float A[],float B[],float C[],int idx[]){
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride)
    {
        if (A[i] >= B[i]) {
            C[i] = A[i];
            idx[i] = 0;
        }
        else {
            C[i] = B[i];
            idx[i] = 1;
        }
    }
}

int main(){
  const size_t ARRAY_SIZE = 1<<24;
  const size_t INT_ARRAY_BYTES = ARRAY_SIZE * sizeof(int);
  const size_t FLOAT_ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
 //number of times the program is to be executed
   const size_t loope = 10;
//declare array
  float *A,*B,*C; 
  int *idx;
  cudaMallocManaged(&A, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&B, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&C, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&idx, INT_ARRAY_BYTES);
// *** init array
  int i;
  for (i = 0; i < ARRAY_SIZE; i++) {
    A[i] = sin(i * 0.0005) * 100.0 + 50.0;
    B[i] = cos(i * 0.0003) * 100.0 + 50.0;
  }

// *** setup CUDA kernel
  size_t numThreads = 1024;
  size_t numBlocks = (ARRAY_SIZE + numThreads-1) / numThreads;

  printf("*** function = Float kernel_C\n");
  printf("numElements = %lu\n", ARRAY_SIZE);
  printf("numBlocks = %lu, numThreads = %lu \n",numBlocks,numThreads);
  for (size_t i=0; i<loope;i++)
    kernel_C <<<numBlocks, numThreads>>> (ARRAY_SIZE,A,B,C,idx);
//barrier
    cudaDeviceSynchronize();
  int sanity_checker = 0;
  for (int i = 0; i < ARRAY_SIZE; i++) { 
        float expected_C = (A[i] >= B[i]) ? A[i] : B[i];
        int expected_idx = (A[i] >= B[i]) ? 0 : 1;
    
        if (fabs(C[i] - expected_C) > 1e-5 || idx[i] != expected_idx) {
            sanity_checker++;
        }
    }
  printf("Sanity Checker = %d",sanity_checker);
//free memory
  cudaFree(A);
  cudaFree(B);
  cudaFree(C);
  cudaFree(idx);
  return 0;
}


Overwriting CUDA_var2.cu


In [8]:
%%bash
nvcc CUDA_var2.cu -o CUDA_var2 -Wno-deprecated-gpu-targets

In [9]:
%%bash
nvprof ./CUDA_var2

==1031626== NVPROF is profiling process 1031626, command: ./CUDA_var2


*** function = Float kernel_C
numElements = 16777216
numBlocks = 16384, numThreads = 1024 
Sanity Checker = 0

==1031626== Profiling application: ./CUDA_var2
==1031626== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  132.99ms        10  13.299ms  343.52us  129.89ms  kernel_C(unsigned long, float*, float*, float*, int*)
      API calls:   91.09%  1.80177s         4  450.44ms  105.07us  1.80057s  cudaMallocManaged
                    6.71%  132.73ms         1  132.73ms  132.73ms  132.73ms  cudaDeviceSynchronize
                    1.93%  38.275ms         4  9.5687ms  8.8571ms  10.552ms  cudaFree
                    0.22%  4.3335ms        10  433.35us  19.137us  4.0328ms  cudaLaunchKernel
                    0.04%  750.17us       114  6.5800us     120ns  288.82us  cuDeviceGetAttribute
                    0.01%  140.62us         1  140.62us  140.62us  140.62us  cuDeviceGetName
                    0.00%  31.837us         1  31.837us  31.837us  31.837us  cuDeviceGetPCIBusId
                    0.00%  29.756us         1  2

# Variant 3.0 - Grid Stride Loop with Prefetch

In [10]:
%%writefile CUDA_var3.cu
// prefetch
#include <stdio.h>
#include <stdlib.h>

//CUDA kernel
__global__
void kernel_C(size_t n, float A[],float B[],float C[],int idx[]){
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride)
    {
        if (A[i] >= B[i]) {
            C[i] = A[i];
            idx[i] = 0;
        }
        else {
            C[i] = B[i];
            idx[i] = 1;
        }
    }
}

int main(){
  const size_t ARRAY_SIZE = 1<<24;
  const size_t INT_ARRAY_BYTES = ARRAY_SIZE * sizeof(int);
  const size_t FLOAT_ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
 //number of times the program is to be executed
   const size_t loope = 10;
//declare array
  float *A,*B,*C; 
  int *idx;
  cudaMallocManaged(&A, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&B, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&C, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&idx, INT_ARRAY_BYTES);

//get gpu id
  int device = -1;
  cudaGetDevice(&device);

// ****init array
  int i;
  for (i = 0; i < ARRAY_SIZE; i++) {
    A[i] = sin(i * 0.0005) * 100.0 + 50.0;
    B[i] = cos(i * 0.0003) * 100.0 + 50.0;
  }
 //"Prefetch data" from CPU-GPU
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,device,NULL);

// setup CUDA kernel
    size_t numThreads = 1024;
    size_t numBlocks = (ARRAY_SIZE + numThreads-1) / numThreads;

  printf("*** function ***\n");
  printf("numElements = %lu\n", ARRAY_SIZE);
  printf("numBlocks = %lu, numThreads = %lu \n",numBlocks, numThreads);
  for (size_t i=0; i<loope;i++)
    kernel_C <<<numBlocks, numThreads>>> (ARRAY_SIZE,A,B,C,idx);
//barrier
    cudaDeviceSynchronize(); 

//"Prefetch data" from GPU-CPU
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,cudaCpuDeviceId,NULL);

  int sanity_checker = 0;
  for (int i = 0; i < ARRAY_SIZE; i++) { 
        float expected_C = (A[i] >= B[i]) ? A[i] : B[i];
        int expected_idx = (A[i] >= B[i]) ? 0 : 1;
    
        if (fabs(C[i] - expected_C) > 1e-5 || idx[i] != expected_idx) {
            sanity_checker++;
        }
    }
  printf("Sanity Checker = %d",sanity_checker);
//free memory
  cudaFree(A);
  cudaFree(B);
  cudaFree(C);
  cudaFree(idx);
  return 0;
}

Overwriting CUDA_var3.cu


In [11]:
%%bash
nvcc CUDA_var3.cu -o CUDA_var3 -Wno-deprecated-gpu-targets

In [12]:
%%bash
nvprof ./CUDA_var3

==1031681== NVPROF is profiling process 1031681, command: ./CUDA_var3


*** function ***
numElements = 16777216
numBlocks = 16384, numThreads = 1024 
Sanity Checker = 0

==1031681== Profiling application: ./CUDA_var3
==1031681== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  3.5414ms        10  354.14us  351.36us  359.26us  kernel_C(unsigned long, float*, float*, float*, int*)
      API calls:   85.45%  1.74109s         4  435.27ms  54.518us  1.74009s  cudaMallocManaged
                   12.21%  248.79ms         8  31.098ms  14.779us  81.622ms  cudaMemPrefetchAsync
                    1.14%  23.296ms         4  5.8239ms  4.7439ms  7.7066ms  cudaFree
                    0.99%  20.152ms        10  2.0152ms  7.8680us  19.984ms  cudaLaunchKernel
                    0.16%  3.3146ms         1  3.3146ms  3.3146ms  3.3146ms  cudaDeviceSynchronize
                    0.02%  506.19us       114  4.4400us     100ns  218.99us  cuDeviceGetAttribute
                    0.01%  243.93us         1  243.93us  243.93us  243.93us  cuDeviceGetName
                    0.01%  108.28us         1  

# Variant 4.0 - Grid Stride Loop with Prefetch and Page Creation

In [13]:
%%writefile CUDA_var4.cu
//prefetch + page creation
// page creation responsible gpu fault page
#include <stdio.h>
#include <stdlib.h>

//CUDA kernel
__global__
void kernel_C(size_t n, float A[],float B[],float C[],int idx[]){
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride)
    {
        if (A[i] >= B[i]) {
            C[i] = A[i];
            idx[i] = 0;
        }
        else {
            C[i] = B[i];
            idx[i] = 1;
        }
    }
}


int main(){
  const size_t ARRAY_SIZE = 1<<24;
  const size_t INT_ARRAY_BYTES = ARRAY_SIZE * sizeof(int);
  const size_t FLOAT_ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
 //number of times the program is to be executed
   const size_t loope = 10;
//declare array
  float *A,*B,*C; 
  int *idx;
  cudaMallocManaged(&A, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&B, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&C, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&idx, INT_ARRAY_BYTES);
//get gpu id
  int device = -1;
  cudaGetDevice(&device);

//"prefetch data" to create CPU page memory
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
//"prefetch data" to create GPU page memory
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,device,NULL);

// ****init array
  int i;
  for (i = 0; i < ARRAY_SIZE; i++) {
    A[i] = sin(i * 0.0005) * 100.0 + 50.0;
    B[i] = cos(i * 0.0003) * 100.0 + 50.0;
  }
//"Prefetch data" from CPU-GPU
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,device,NULL);

// setup CUDA kernel
    size_t numThreads = 1024;   
    size_t numBlocks = (ARRAY_SIZE + numThreads-1) / numThreads;

  printf("*** function ***\n");
  printf("numElements = %lu\n", ARRAY_SIZE);
  printf("numBlocks = %lu, numThreads = %lu \n",numBlocks, numThreads);
  for (size_t i=0; i<loope;i++)
    kernel_C <<<numBlocks, numThreads>>> (ARRAY_SIZE,A,B,C,idx);
//barrier
    cudaDeviceSynchronize();

  //"Prefetch data" from GPU-CPU
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,cudaCpuDeviceId,NULL);

  int sanity_checker = 0;
  for (int i = 0; i < ARRAY_SIZE; i++) { 
        float expected_C = (A[i] >= B[i]) ? A[i] : B[i];
        int expected_idx = (A[i] >= B[i]) ? 0 : 1;
    
        if (fabs(C[i] - expected_C) > 1e-5 || idx[i] != expected_idx) {
            sanity_checker++;
        }
    }
  printf("Sanity Checker = %d",sanity_checker);

//free memory
  cudaFree(A);
  cudaFree(B);
  cudaFree(C);
  cudaFree(idx);
  return 0;
}

Overwriting CUDA_var4.cu


In [14]:
%%bash
nvcc CUDA_var4.cu -o CUDA_var4 -Wno-deprecated-gpu-targets

In [15]:
%%bash
nvprof ./CUDA_var4

==1031742== NVPROF is profiling process 1031742, command: ./CUDA_var4


*** function ***
numElements = 16777216
numBlocks = 16384, numThreads = 1024 
Sanity Checker = 0

==1031742== Profiling application: ./CUDA_var4
==1031742== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  3.6435ms        10  364.35us  361.57us  367.20us  kernel_C(unsigned long, float*, float*, float*, int*)
      API calls:   82.78%  1.94579s         4  486.45ms  77.617us  1.94460s  cudaMallocManaged
                   14.97%  351.87ms        12  29.322ms  24.799us  72.411ms  cudaMemPrefetchAsync
                    1.44%  33.910ms         4  8.4776ms  8.2545ms  8.7662ms  cudaFree
                    0.62%  14.462ms        10  1.4462ms  12.599us  14.267ms  cudaLaunchKernel
                    0.15%  3.5585ms         1  3.5585ms  3.5585ms  3.5585ms  cudaDeviceSynchronize
                    0.03%  769.78us       114  6.7520us     133ns  344.23us  cuDeviceGetAttribute
                    0.01%  163.05us         1  163.05us  163.05us  163.05us  cuDeviceGetName
                    0.00%  28.526us         1  

# Variant 5.0 - Grid Stride Loop with Prefetch and Page Creation + mem advise

In [16]:
%%writefile CUDA_var5.cu
//prefetch + page creation + memadvise

#include <stdio.h>
#include <stdlib.h>

//CUDA kernel
__global__
void kernel_C(size_t n, float A[],float B[],float C[],int idx[]){
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride)
    {
        if (A[i] >= B[i]) {
            C[i] = A[i];
            idx[i] = 0;
        }
        else {
            C[i] = B[i];
            idx[i] = 1;
        }
    }
}

int main(){
  const size_t ARRAY_SIZE = 1<<24;
  const size_t INT_ARRAY_BYTES = ARRAY_SIZE * sizeof(int);
  const size_t FLOAT_ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
 //number of times the program is to be executed
   const size_t loope = 10;
//declare array
  float *A,*B,*C; 
  int *idx;
  cudaMallocManaged(&A, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&B, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&C, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&idx, INT_ARRAY_BYTES);

//get gpu id
  int device = -1;
  cudaGetDevice(&device);

// memory advise
   cudaMemAdvise(A, FLOAT_ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
   cudaMemAdvise(A, FLOAT_ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);
   cudaMemAdvise(B, FLOAT_ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
   cudaMemAdvise(B, FLOAT_ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);

//"prefetch data" to create CPU page memory
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
//"prefetch data" to create GPU page memory
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,device,NULL);

// ****init array
  int i;
  for (i = 0; i < ARRAY_SIZE; i++) {
    A[i] = sin(i * 0.0005) * 100.0 + 50.0;
    B[i] = cos(i * 0.0003) * 100.0 + 50.0;
  }


 //"Prefetch data" from CPU-GPU
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,device,NULL);

// setup CUDA kernel
    size_t numThreads = 1024;   // what if numThreads>1024, numThreads<1024?
    size_t numBlocks = (ARRAY_SIZE + numThreads-1) / numThreads;

  printf("*** function ***\n");
  printf("numElements = %lu\n", ARRAY_SIZE);
  printf("numBlocks = %lu, numThreads = %lu \n",numBlocks, numThreads);
  for (size_t i=0; i<loope;i++)
    kernel_C <<<numBlocks, numThreads>>> (ARRAY_SIZE,A,B,C,idx);
//barrier
    cudaDeviceSynchronize();

  //"Prefetch data" from GPU-CPU
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,cudaCpuDeviceId,NULL);


  int sanity_checker = 0;
  for (int i = 0; i < ARRAY_SIZE; i++) { 
        float expected_C = (A[i] >= B[i]) ? A[i] : B[i];
        int expected_idx = (A[i] >= B[i]) ? 0 : 1;
    
        if (fabs(C[i] - expected_C) > 1e-5 || idx[i] != expected_idx) {
            sanity_checker++;
        }
    }
  printf("Sanity Checker = %d",sanity_checker);

//free memory
  cudaFree(A);
  cudaFree(B);
  cudaFree(C);
  cudaFree(idx);
  return 0;

}


Overwriting CUDA_var5.cu


In [17]:
%%bash
nvcc CUDA_var5.cu -o CUDA_var5 -Wno-deprecated-gpu-targets

In [18]:
%%bash
nvprof ./CUDA_var5

==1031799== NVPROF is profiling process 1031799, command: ./CUDA_var5


*** function ***
numElements = 16777216
numBlocks = 16384, numThreads = 1024 
Sanity Checker = 0

==1031799== Profiling application: ./CUDA_var5
==1031799== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  3.6357ms        10  363.57us  361.79us  367.42us  kernel_C(unsigned long, float*, float*, float*, int*)
      API calls:   84.08%  1.88468s         4  471.17ms  70.095us  1.88343s  cudaMallocManaged
                   14.47%  324.33ms        12  27.028ms  240.09us  80.249ms  cudaMemPrefetchAsync
                    1.00%  22.419ms         4  5.6047ms  3.4407ms  7.6605ms  cudaFree
                    0.15%  3.4218ms        10  342.18us  10.756us  3.1489ms  cudaLaunchKernel
                    0.15%  3.3064ms         1  3.3064ms  3.3064ms  3.3064ms  cudaDeviceSynchronize
                    0.10%  2.1965ms       114  19.267us     162ns  949.78us  cuDeviceGetAttribute
                    0.02%  508.80us         4  127.20us  5.6700us  464.77us  cudaMemAdvise
                    0.02%  498.15us         1  49

# Variant 6.0 - CUDA classic MEMCPY

In [19]:
%%writefile CUDA_var6.cu
//grid stride loop + memcpy
// BFF PALANG TOH SO FAR + onting research sa websites HJSHAS
// UNFINISHED

#include <stdio.h>
#include <stdlib.h>
#include <math.h>

//CUDA kernel
__global__
void kernel_C(size_t n, float A[],float B[],float C[],int idx[]){
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride)
    {
        if (A[i] >= B[i]) {
            C[i] = A[i];
            idx[i] = 0;
        }
        else {
            C[i] = B[i];
            idx[i] = 1;
        }
    }
}

int main(){
  const size_t ARRAY_SIZE = 1<<24;
  const size_t INT_ARRAY_BYTES = ARRAY_SIZE * sizeof(int);
  const size_t FLOAT_ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
 //number of times the program is to be executed
   const size_t loope = 10;

float *h_A = (float*)malloc(FLOAT_ARRAY_BYTES);
float *h_B = (float*)malloc(FLOAT_ARRAY_BYTES);
float *h_C = (float*)malloc(FLOAT_ARRAY_BYTES);
int *h_idx = (int*)malloc(INT_ARRAY_BYTES);

// ****init host array
  int i;
  for (i = 0; i < ARRAY_SIZE; i++) {
    h_A[i] = sinf(i * 0.0005) * 100.0 + 50.0;
    h_B[i] = cosf(i * 0.0003) * 100.0 + 50.0;
  }

// device allocations
  float *d_A,*d_B,*d_C;
  int *d_idx;
  cudaMalloc(&d_A, FLOAT_ARRAY_BYTES); //we dont use cudaMallocManaged with cudaMemcpy
  cudaMalloc(&d_B, FLOAT_ARRAY_BYTES);
  cudaMalloc(&d_C, FLOAT_ARRAY_BYTES);
  cudaMalloc(&d_idx, INT_ARRAY_BYTES);

//get gpu id
  int device = -1;
  cudaGetDevice(&device);

// Copy data to device
    cudaMemcpy(d_A, h_A, FLOAT_ARRAY_BYTES, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, FLOAT_ARRAY_BYTES, cudaMemcpyHostToDevice);

// Kernel launch configuration
    int threadsPerBlock = 256;
    int blocksPerGrid = (ARRAY_SIZE + threadsPerBlock - 1) / threadsPerBlock;
    
  for (size_t i=0; i<loope;i++){
    kernel_C <<<blocksPerGrid, threadsPerBlock>>> (ARRAY_SIZE, d_A, d_B, d_C, d_idx);
    cudaDeviceSynchronize();
  }

    
// Copy results back to host
    cudaMemcpy(h_C, d_C, FLOAT_ARRAY_BYTES, cudaMemcpyDeviceToHost);
    cudaMemcpy(h_idx, d_idx, INT_ARRAY_BYTES, cudaMemcpyDeviceToHost);

    // Verification loop
    int errors = 0;
    for (int i = 0; i < ARRAY_SIZE; i++) { 
        float expected_C = (h_A[i] >= h_B[i]) ? h_A[i] : h_B[i];
        int expected_idx = (h_A[i] >= h_B[i]) ? 0 : 1;
    
        if (fabs(h_C[i] - expected_C) > 1e-5 || h_idx[i] != expected_idx) {
            errors++;
        }
    }
    printf("Errors = %d", errors);

// Free device memory
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
    cudaFree(d_idx);

// Free host memory
    free(h_A);
    free(h_B);
    free(h_C);
    free(h_idx);

  return 0;
}

Overwriting CUDA_var6.cu


In [20]:
%%bash
nvcc CUDA_var6.cu -o CUDA_var6 -Wno-deprecated-gpu-targets

In [21]:
%%bash
nvprof ./CUDA_var6

==1031854== NVPROF is profiling process 1031854, command: ./CUDA_var6


Errors = 0

==1031854== Profiling application: ./CUDA_var6
==1031854== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   81.33%  781.32ms         2  390.66ms  385.76ms  395.56ms  [CUDA memcpy DtoH]
                   18.31%  175.87ms         2  87.936ms  83.615ms  92.258ms  [CUDA memcpy HtoD]
                    0.36%  3.4481ms        10  344.81us  343.42us  348.35us  kernel_C(unsigned long, float*, float*, float*, int*)
      API calls:   65.95%  1.93466s         4  483.66ms  1.4087ms  1.92970s  cudaMalloc
                   33.55%  984.19ms         4  246.05ms  85.231ms  408.31ms  cudaMemcpy
                    0.22%  6.5389ms        10  653.89us  278.12us  3.5259ms  cudaDeviceSynchronize
                    0.13%  3.7010ms        10  370.10us  17.592us  2.7638ms  cudaLaunchKernel
                    0.11%  3.1907ms         4  797.68us  246.61us  2.2443ms  cudaFree
                    0.03%  852.31us       114  7.4760us     141

# Variant 7.0 - CUDA init

In [22]:
%%writefile CUDA_var7.cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

// CUDA kernel to initialize A and B
__global__
void init_arrays(size_t n, float *A, float *B) {
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride) {
        A[i] = sinf(i * 0.0005f) * 100.0f + 50.0f;
        B[i] = cosf(i * 0.0003f) * 100.0f + 50.0f;
    }
}

int main() {
    const size_t ARRAY_SIZE = 1 << 24;
    const size_t FLOAT_BYTES = ARRAY_SIZE * sizeof(float);

    // Unified memory allocation
    float *A, *B;
    cudaMallocManaged(&A, FLOAT_BYTES);
    cudaMallocManaged(&B, FLOAT_BYTES);


    // Launch kernel
    int threads = 256;
    int blocks = (ARRAY_SIZE + threads - 1) / threads;
    init_arrays<<<blocks, threads>>>(ARRAY_SIZE, A, B);
    cudaDeviceSynchronize();
    
    // Verification loop (no memcpy needed!)
    int error = 0;
    for (int i = 0; i < 100; i++) {
        float expected_A = sinf(i * 0.0005f) * 100.0f + 50.0f;
        float expected_B = cosf(i * 0.0003f) * 100.0f + 50.0f;
        if (fabs(A[i] - expected_A) > 1e-4f || fabs(B[i] - expected_B) > 1e-4f) {
            error++;
        }
    }
    printf("Error = %d\n", error);
    
    // Free unified memory
    cudaFree(A);
    cudaFree(B);

    return 0;
}

Overwriting CUDA_var7.cu


In [23]:
%%bash
nvcc CUDA_var7.cu -o CUDA_var7 -Wno-deprecated-gpu-targets

In [24]:
%%bash
nvprof ./CUDA_var7

==1031909== NVPROF is profiling process 1031909, command: ./CUDA_var7


Error = 0


==1031909== Profiling application: ./CUDA_var7
==1031909== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  46.889ms         1  46.889ms  46.889ms  46.889ms  init_arrays(unsigned long, float*, float*)
      API calls:   97.13%  1.82120s         2  910.60ms  896.45us  1.82030s  cudaMallocManaged
                    2.50%  46.945ms         1  46.945ms  46.945ms  46.945ms  cudaDeviceSynchronize
                    0.17%  3.1867ms         2  1.5933ms  1.2385ms  1.9481ms  cudaFree
                    0.15%  2.8700ms         1  2.8700ms  2.8700ms  2.8700ms  cudaLaunchKernel
                    0.03%  585.35us       114  5.1340us     131ns  205.68us  cuDeviceGetAttribute
                    0.01%  209.90us         1  209.90us  209.90us  209.90us  cuDeviceGetName
                    0.00%  47.214us         1  47.214us  47.214us  47.214us  cuDeviceTotalMem
                    0.00%  15.650us         1  15.650us  15.65